<a href="https://colab.research.google.com/github/gabrielhozana/Cheesman-Model-Using-Transfer-Learning-ResNet152V2/blob/main/Cheesman_Model_Using_Transfer_Learning_ResNet152V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget --no-check-certificate \
  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pengembangan_academy/Chessman-image-dataset.zip \
  -O /tmp/Chessman-image-dataset.zip

--2021-06-29 01:38:49--  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pengembangan_academy/Chessman-image-dataset.zip
Resolving dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)... 52.239.197.36
Connecting to dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)|52.239.197.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60684125 (58M) [application/zip]
Saving to: ‘/tmp/Chessman-image-dataset.zip’

/tmp/Chessman-image 100%[===================>]  57.87M  5.17MB/s    in 13s     

2021-06-29 01:39:03 (4.49 MB/s) - ‘/tmp/Chessman-image-dataset.zip’ saved [60684125/60684125]



In [2]:
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
local_zip = '/tmp/Chessman-image-dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()
train_dir = os.path.join('/tmp/Chessman-image-dataset/Chess')
#membuat augmentasi 
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   zoom_range=0.2,
                                   shear_range=0.2,
                                   fill_mode = 'nearest',
                                   validation_split=0.1) # set validation split

In [3]:
#Bagi dataset menjadi data training dan data validasi.
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=8,
    class_mode='categorical',
    subset='training') # set as training data
validation_generator = train_datagen.flow_from_directory(
    train_dir, # same directory as training data
    target_size=(150, 150),
    batch_size=16,
    class_mode='categorical',
    subset='validation')

Found 499 images belonging to 6 classes.
Found 52 images belonging to 6 classes.


In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import ResNet152V2
model = tf.keras.models.Sequential([
                                    #menggunakan model ResNet152V2 untuk transfer learning
                                    #weights: kita ingin menggunakan model ResNet152V2 yang telah dilatih pada dataset imagenet.
                                    #Include_top: Maksud dari parameter ini apabila kita ingin tetap memakai layer terakhir/layer prediksi dari model resnet. Kita isi false karena kita memakai model resnet untuk memprediksi dataset chessman bukan imagenet.
                                    #Input_tensor : sesuai namanya parameter ini menspesifikasikan ukuran dari input.
                                    ResNet152V2(weights="imagenet", include_top=False, input_tensor=Input(shape=(150, 150, 3))),
                                    # tf.keras.layers.Dropout(0.4),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512, activation='relu'),
                                    tf.keras.layers.Dense(256, activation='relu'),
                                    tf.keras.layers.Dense(6, activation='softmax')
                                    ])
model.layers[0].trainable = False

In [5]:
#menentukan optimizer, loss dan metrik
model.compile(optimizer=tf.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics = ['accuracy'])

In [6]:
history = model.fit(train_generator,
                    validation_data=validation_generator,
                    epochs=50,
                    verbose=2)

Epoch 1/50
63/63 - 106s - loss: 8.7658 - accuracy: 0.4208 - val_loss: 3.5285 - val_accuracy: 0.5192
Epoch 2/50
63/63 - 95s - loss: 1.6916 - accuracy: 0.6593 - val_loss: 0.9051 - val_accuracy: 0.6923
Epoch 3/50
63/63 - 95s - loss: 0.8626 - accuracy: 0.7876 - val_loss: 1.7091 - val_accuracy: 0.5962
Epoch 4/50
63/63 - 95s - loss: 0.7003 - accuracy: 0.8016 - val_loss: 2.5608 - val_accuracy: 0.5192
Epoch 5/50
63/63 - 95s - loss: 0.7423 - accuracy: 0.8176 - val_loss: 1.1268 - val_accuracy: 0.7500
Epoch 6/50
63/63 - 95s - loss: 0.7494 - accuracy: 0.8036 - val_loss: 0.8826 - val_accuracy: 0.7115
Epoch 7/50
63/63 - 95s - loss: 0.6293 - accuracy: 0.8417 - val_loss: 1.4840 - val_accuracy: 0.6154
Epoch 8/50
63/63 - 95s - loss: 0.5264 - accuracy: 0.8457 - val_loss: 0.7512 - val_accuracy: 0.8077
Epoch 9/50
63/63 - 96s - loss: 0.3564 - accuracy: 0.9038 - val_loss: 1.9136 - val_accuracy: 0.6154
Epoch 10/50
63/63 - 95s - loss: 0.5038 - accuracy: 0.8758 - val_loss: 0.9579 - val_accuracy: 0.7115
Epoch 11